**Stage 1 - Training**

In [ ]:
# @title 1. Setup & Dependencies
import os
import sys
import subprocess

# 1.1 Mount Drive
from google.colab import drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# 1.2 Project Paths
PROJECT_ROOT = '/content/drive/MyDrive/projects/EarthShader'
DATASET_DIR = os.path.join(PROJECT_ROOT, 'dataset/stage1')
CHECKPOINT_DIR = os.path.join(PROJECT_ROOT, 'checkpoints/stage1_adapter')
LOG_DIR = os.path.join(PROJECT_ROOT, 'logs')

os.makedirs(CHECKPOINT_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

# 1.3 Install Standard HF Libraries
# We need the latest transformers for Qwen2-VL support
print("Installing Hugging Face Libraries...")
packages = [
    "git+https://github.com/huggingface/transformers",
    "peft",
    "datasets",
    "bitsandbytes",
    "accelerate",
    "qwen-vl-utils",
    "trl"
]
subprocess.check_call([sys.executable, "-m", "pip", "install"] + packages)

print("Environment Ready.")

In [ ]:
# @title 2. Download Model (Stable Version)
import os
import shutil
from huggingface_hub import snapshot_download

# 1. Clean up the broken 2.5 download to free up space
if os.path.exists("/content/qwen_local"):
    print("Cleaning up incompatible model files...")
    shutil.rmtree("/content/qwen_local")

# 2. Download the Stable Qwen2-VL (Not 2.5)
# This version is fully compatible with the current transformers library
MODEL_ID = "Qwen/Qwen2-VL-7B-Instruct"

print(f"Downloading {MODEL_ID}...")
local_model_path = snapshot_download(
    repo_id=MODEL_ID,
    local_dir="/content/qwen_local",
    local_dir_use_symlinks=False,
    resume_download=True
)

print(f"Model ready at: {local_model_path}")

In [ ]:
# @title 3. Load Model (Corrected & Standardized)
import torch
from transformers import Qwen2VLForConditionalGeneration, Qwen2VLProcessor, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType

MODEL_PATH = "/content/qwen_local"

# --- RESOLUTION SETTINGS ---
# We calculate total pixels based on target resolution (Width x Height)
# 1. Min: 224x224 (Standard minimal size for ViTs)
MIN_PIXELS = 224 * 224

# 2. Max: 256x256 (Aggressive cap to save VRAM on T4)
MAX_PIXELS = 256 * 256

print(f"Loading Processor with:")
print(f" - Min Resolution: 224x224 ({MIN_PIXELS} pixels)")
print(f" - Max Resolution: 256x256 ({MAX_PIXELS} pixels)")

processor = Qwen2VLProcessor.from_pretrained(
    MODEL_PATH,
    min_pixels=MIN_PIXELS,
    max_pixels=MAX_PIXELS
)

# 2. Load Base Model (4-bit)
BNB_CONFIG = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

model = Qwen2VLForConditionalGeneration.from_pretrained(
    MODEL_PATH,
    quantization_config=BNB_CONFIG,
    device_map={"": 0},
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
)

# 3. Apply LoRA
model.gradient_checkpointing_enable()

lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    modules_to_save=[],
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
model.enable_input_require_grads()

In [ ]:
# @title 4. Dataset & DataLoader
import json
import os
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image

# 1. Define Paths (Self-Contained)
PROJECT_ROOT = '/content/drive/MyDrive/projects/EarthShader'
DATASET_DIR = os.path.join(PROJECT_ROOT, 'dataset/stage1')

# 2. Define Dataset Class
class ShaderDataset(Dataset):
    def __init__(self, jsonl_path):
        self.samples = []
        if not os.path.exists(jsonl_path):
            print(f"Error: {jsonl_path} not found.")
            return

        with open(jsonl_path, 'r') as f:
            for line in f:
                try:
                    entry = json.loads(line)
                    if os.path.exists(entry['image_path']):
                        self.samples.append(entry)
                except:
                    continue
        print(f"Loaded {len(self.samples)} valid samples.")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]

# 3. Define Collate Function
def collate_fn(batch):
    images = []
    texts = []

    for item in batch:
        # Load Image on the fly to save RAM
        try:
            image = Image.open(item['image_path']).convert("RGB")
        except:
            image = Image.new("RGB", (256, 256), (0, 0, 0)) # Fallback

        images.append(image)

        # Construct ChatML Prompt
        analysis = item['analysis']
        code = item['code']
        full_response = f"{analysis}\n\n// GLSL CODE\n{code}"

        # Standard Qwen2-VL Prompt Format
        conversation = [
            {
                "role": "user",
                "content": [
                    {"type": "image"},
                    {"type": "text", "text": "Reverse engineer the GLSL shader code for this texture. Include analysis."}
                ]
            },
            {
                "role": "assistant",
                "content": [{"type": "text", "text": full_response}]
            }
        ]

        # Apply template using the processor
        text_prompt = processor.apply_chat_template(conversation, tokenize=False, add_generation_prompt=False)
        texts.append(text_prompt)

    # Process Batch
    inputs = processor(
        text=texts,
        images=images,
        padding=True,
        return_tensors="pt",
    )

    # Create Labels
    inputs["labels"] = inputs["input_ids"].clone()

    # Mask padding
    if processor.tokenizer.pad_token_id is not None:
        inputs["labels"][inputs["input_ids"] == processor.tokenizer.pad_token_id] = -100

    return inputs

# 4. Initialize Loader
jsonl_file = os.path.join(DATASET_DIR, 'dataset.jsonl')
dataset = ShaderDataset(jsonl_file)

train_dataloader = DataLoader(
    dataset,
    batch_size=1,       # T4 Limit
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=2,      # Pre-load images in background
    pin_memory=True
)

print("DataLoader ready.")

In [ ]:
# @title 5. Speed Test (Fixed Masking & LR)
from torch.optim import AdamW
import bitsandbytes as bnb
from tqdm import tqdm
import torch
import gc

# 1. Shrink Dataset (Overfit Test)
mini_dataset = torch.utils.data.Subset(dataset, range(128))

# 2. Collator with LABEL MASKING (Critical Fix)
def masked_collate_fn(batch):
    # Standard processing first
    inputs = collate_fn(batch)

    # Now mask the user prompts so we don't train on them
    # Qwen uses <|im_start|>assistant as the trigger
    # We find the index of the answer start and mask everything before it
    input_ids = inputs["input_ids"]
    labels = inputs["labels"]

    # Qwen token IDs (approximate, scanning for standard chat template structure)
    # A robust way is to just mask the first N tokens if length is consistent,
    # but dynamic masking is safer.
    for i in range(len(input_ids)):
        # Find the start of the assistant's response
        # This is a heuristic: finding the last instance of the User/System sequence
        # For simplicity in this test: Mask the first 50 tokens (The visual + prompt header)
        # Real Qwen prompt is roughly 40-60 tokens long + image tokens
        labels[i, :64] = -100 # Ignore the image and the question

    return inputs

# 3. Loader
mini_loader = DataLoader(
    mini_dataset,
    batch_size=1,
    shuffle=True,
    collate_fn=masked_collate_fn, # Use new masker
    num_workers=2,
    pin_memory=True
)

# 4. Configuration
EPOCHS = 10             # More epochs to force convergence
GRAD_ACCUMULATION = 8
LEARNING_RATE = 5e-5    # Lower LR for stability

# 5. Optimizer Reset
params = [p for p in model.parameters() if p.requires_grad]
optimizer = bnb.optim.PagedAdamW8bit(params, lr=LEARNING_RATE)

# 6. Loop
model.train()
print(f"🚀 Starting MASKED Sanity Check...")
print("Target Loss: < 1.0")

gc.collect()
torch.cuda.empty_cache()

for epoch in range(EPOCHS):
    progress_bar = tqdm(mini_loader, desc=f"Epoch {epoch+1}/{EPOCHS}")
    total_loss = 0
    steps = 0

    for step, batch in enumerate(progress_bar):
        try:
            batch = {k: v.to(model.device) for k, v in batch.items()}

            outputs = model(**batch, use_cache=False)
            loss = outputs.loss / GRAD_ACCUMULATION
            loss.backward()

            total_loss += outputs.loss.item()
            steps += 1

            if (step + 1) % GRAD_ACCUMULATION == 0:
                optimizer.step()
                optimizer.zero_grad()

                avg_loss = total_loss / steps
                progress_bar.set_postfix({'loss': f'{avg_loss:.4f}'})

        except RuntimeError as e:
            if "out of memory" in str(e):
                optimizer.zero_grad()
                torch.cuda.empty_cache()
            else:
                raise e

In [ ]:
# @title 5. Full Training Run
from torch.optim import AdamW
import bitsandbytes as bnb
from tqdm import tqdm
import torch
import gc
import os
from PIL import Image

# 1. Configuration
EPOCHS = 1
GRAD_ACCUMULATION = 8
LEARNING_RATE = 2e-4
SAVE_STEPS = 500
MAX_LENGTH = 512

# 2. Define Collator (With Text Truncation)
def smart_collate_fn(batch):
    images = []
    full_texts = []
    prompt_only_texts = []

    for item in batch:
        # Load Image
        try:
            image = Image.open(item['image_path']).convert("RGB")
        except:
            image = Image.new("RGB", (256, 256), (0, 0, 0))
        images.append(image)

        # Prepare Texts
        analysis = item['analysis']
        code = item['code']
        full_response = f"{analysis}\n\n// GLSL CODE\n{code}"

        # Prompt Only
        conversation_prompt = [
            {
                "role": "user",
                "content": [
                    {"type": "image"},
                    {"type": "text", "text": "Reverse engineer the GLSL shader code for this texture. Include analysis."}
                ]
            }
        ]
        prompt_str = processor.apply_chat_template(conversation_prompt, tokenize=False, add_generation_prompt=True)

        # Full Conversation
        conversation_full = [
            {
                "role": "user",
                "content": [
                    {"type": "image"},
                    {"type": "text", "text": "Reverse engineer the GLSL shader code for this texture. Include analysis."}
                ]
            },
            {
                "role": "assistant",
                "content": [{"type": "text", "text": full_response}]
            }
        ]
        full_str = processor.apply_chat_template(conversation_full, tokenize=False, add_generation_prompt=False)

        prompt_only_texts.append(prompt_str)
        full_texts.append(full_str)

    # Tokenize with HARD CAPS
    inputs = processor(
        text=full_texts,
        images=images,
        padding="max_length", # Consistent tensor size
        max_length=MAX_LENGTH, # Cap at 512
        truncation=True,       # Cut off if longer
        return_tensors="pt",
    )

    # We need a separate pass for prompts to calculate masking length
    # We use the same caps to ensure indices match
    inputs_prompts = processor(
        text=prompt_only_texts,
        images=images,
        padding="max_length",
        max_length=MAX_LENGTH,
        truncation=True,
        return_tensors="pt",
    )

    # Masking
    labels = inputs["input_ids"].clone()
    for i in range(len(batch)):
        # Calculate prompt length
        prompt_len = inputs_prompts["attention_mask"][i].sum().item()

        # Safety clamp
        prompt_len = min(prompt_len, MAX_LENGTH)

        # Mask prompt
        labels[i, :prompt_len] = -100

        # Mask padding (tokens equal to pad_token_id)
        if processor.tokenizer.pad_token_id is not None:
            labels[i][inputs["input_ids"][i] == processor.tokenizer.pad_token_id] = -100

    inputs["labels"] = labels
    return inputs

# 3. Create Full Loader
full_loader = DataLoader(
    dataset,
    batch_size=1,
    shuffle=True,
    collate_fn=smart_collate_fn,
    num_workers=2,
    pin_memory=True
)

# 4. Optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = bnb.optim.PagedAdamW8bit(params, lr=LEARNING_RATE)

# 5. Resume Logic
global_step = 0
if os.path.exists(CHECKPOINT_DIR):
    checkpoints = [d for d in os.listdir(CHECKPOINT_DIR) if d.startswith("checkpoint")]
    if checkpoints:
        checkpoints.sort(key=lambda x: int(x.split('-')[-1]))
        print(f"  Resuming from epoch 0 (clean start recommended for Stage 1)")

# 6. Training Loop
model.train()
print(f"  Starting FULL TRAINING (Max Len={MAX_LENGTH})")

gc.collect()
torch.cuda.empty_cache()

for epoch in range(EPOCHS):
    progress_bar = tqdm(full_loader, desc=f"Epoch {epoch+1}/{EPOCHS}")
    total_loss = 0
    steps = 0

    for step, batch in enumerate(progress_bar):
        try:
            # Move to GPU
            batch = {k: v.to(model.device) for k, v in batch.items()}

            # Forward
            outputs = model(**batch, use_cache=False)
            loss = outputs.loss / GRAD_ACCUMULATION
            loss.backward()

            total_loss += outputs.loss.item()
            steps += 1

            # Update Step
            if (step + 1) % GRAD_ACCUMULATION == 0:
                optimizer.step()
                optimizer.zero_grad()
                global_step += 1

                # Logging
                avg_loss = total_loss / steps
                progress_bar.set_postfix({'loss': f'{avg_loss:.4f}'})

                # Saving
                if global_step % SAVE_STEPS == 0:
                    save_path = os.path.join(CHECKPOINT_DIR, f"checkpoint-{global_step}")
                    print(f"\nSaving checkpoint: {save_path}")
                    model.save_pretrained(save_path)
                    processor.save_pretrained(save_path)

                    # Cleanup
                    gc.collect()
                    torch.cuda.empty_cache()

        except RuntimeError as e:
            if "out of memory" in str(e):
                print(f"OOM at step {step}. Recovering...")
                optimizer.zero_grad()
                torch.cuda.empty_cache()
            else:
                raise e

# 7. Final Save
final_path = os.path.join(PROJECT_ROOT, "checkpoints/stage1_final")
print(f"\nSaving FINAL model to {final_path}")
model.save_pretrained(final_path)
processor.save_pretrained(final_path)
print("Training Complete.")

In [ ]:
# @title 6. Auto-Shutdown
# This cell will only run after the training cell finishes.
import time
from google.colab import runtime

print("Training finished. Saving is complete.")
print("Shutting down runtime to save Compute Units in 60 seconds...")

# Give time for the final logs to sync to Drive
time.sleep(60)

print("Goodnight.")
runtime.unassign()